# 3D classification example based on DenseNet

This tutorial shows an example of 3D classification task based on DenseNet and array format transforms.

Here, the task is given to classify MR images into male/female.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/main/3d_classification/torch/densenet_training_array.ipynb)

## Setup environment

In [ ]:
#!python -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm]"


## Setup imports

In [ ]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import logging
import os
import glob
import sys
import shutil
import tempfile
import datetime
import socket
import functools

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nibabel as nib

import torch
from torch.utils.tensorboard import SummaryWriter

from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, roc_auc_score

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import DataLoader, ImageDataset
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    RandRotate90,
    Resize,
    ScaleIntensity,
)

pin_memory = torch.cuda.is_available()
torch.backends.cudnn.benchmark = False #torch.cuda.is_available() # Set this to true if the code fails
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()

In [ ]:
def get_predicted_values(model, loader):
    t_model_outputs = []
    t_test_labels = []
    for test_data in loader:
        test_images, test_labels = test_data[0].to(device), test_data[1].to(device)
        with torch.no_grad():
            val_outputs = model(test_images)
            val_outputs_argmax = val_outputs.argmax(dim=1)
            t_model_outputs.append(val_outputs_argmax.cpu().detach().numpy())
            t_test_labels.append(test_labels.cpu().detach().numpy())
    conf_model_outputs = np.concatenate(t_model_outputs)
    conf_test_labels = np.concatenate(t_test_labels)
    return conf_model_outputs, conf_test_labels

def get_cm(conf_model_outputs, conf_test_labels, num_classes):
    conf_matrix = np.zeros((num_classes,num_classes))
    for i in range(num_classes):
        in_class_i = conf_test_labels==i
        for j in range(num_classes):
            in_class_i_predicted_in_class_j = sum(conf_model_outputs[in_class_i]==j)
            conf_matrix[i,j] = in_class_i_predicted_in_class_j
    return conf_matrix

In [ ]:
# Set data directory
#directory = os.environ.get("MONAI_DATA_DIRECTORY")
#root_dir = tempfile.mkdtemp() if directory is None else directory
root_dir = os.path.expanduser(os.path.expandvars("~/data/medicaldecathlon/"))
data_dir = os.path.join(root_dir, "Task10_Colon")
train_dataset_frailty_path = os.path.join(data_dir,"train_v2_clean.csv")
test_dataset_frailty_path = os.path.join(data_dir,"test_v2_clean.csv")
print(root_dir)

In [ ]:
train_images = sorted(
    glob.glob(os.path.join(data_dir, "imagesTr", "*.nii.gz")))
train_images[:5]

In [ ]:
test_image_paths = sorted(
    glob.glob(os.path.join(data_dir, "imagesTs", "*.nii.gz")))
test_image_paths[:5]

In [ ]:
df_labels = pd.read_csv(train_dataset_frailty_path, index_col="PatientID").dropna()
df_labels.head()

In [ ]:
df_labels_test = pd.read_csv(test_dataset_frailty_path, index_col="PatientID").dropna()
df_labels_test.head()

In [ ]:
df_labels.loc[df_labels["Risk Category"]=="LOW","Risk Category"] = 0
df_labels.loc[df_labels["Risk Category"]=="MEDIUM","Risk Category"] = 0
df_labels.loc[df_labels["Risk Category"]=="HIGH","Risk Category"] = 1

df_labels_test.loc[df_labels_test["Risk Category"]=="LOW","Risk Category"] = 0
df_labels_test.loc[df_labels_test["Risk Category"]=="MEDIUM","Risk Category"] = 0
df_labels_test.loc[df_labels_test["Risk Category"]=="HIGH","Risk Category"] = 1

In [ ]:
def get_id_from_filepath(fpath):
    return int(os.path.basename(fpath).split("_")[1].split(".")[0])

In [ ]:
MIN_SLICES = 64

In [ ]:
data_dicts = [
    {"image": image_name, "label": df_labels.loc[patient_id,"Risk Category"]}
    for image_name,patient_id in zip(train_images,map(get_id_from_filepath,train_images))
    if patient_id in df_labels.index and nib.load(image_name).get_fdata().shape[2]>=MIN_SLICES
]

In [ ]:
test_data_dicts = [
    {"image": image_name, "label": df_labels_test.loc[patient_id,"Risk Category"]}
    for image_name,patient_id in zip(test_image_paths,map(get_id_from_filepath,test_image_paths))
    if patient_id in df_labels_test.index and nib.load(image_name).get_fdata().shape[2]>=MIN_SLICES
]
len(test_data_dicts)

In [ ]:
# IXI dataset as a demo, downloadable from https://brain-development.org/ixi-dataset/
images = np.array([d["image"] for d in data_dicts])
# 2 binary labels for gender classification: man or woman
labels = np.array([d["label"] for d in data_dicts])

test_images = [d["image"] for d in test_data_dicts]
# 2 binary labels for gender classification: man or woman
test_labels = [d["label"] for d in test_data_dicts]


# Represent labels in one-hot format for binary classifier training,
# BCEWithLogitsLoss requires target to have same shape as input
#labels = torch.nn.functional.one_hot(torch.as_tensor(labels)).float()
num_labels = int(max(labels))+1
num_labels, labels[:5], test_labels[:5]

In [ ]:
shapes = [nib.load(datapoint["image"]).get_fdata().shape for datapoint in data_dicts]
np_shapes = np.stack(shapes)
minshapes = np.min(np_shapes, axis=0)
minshapes

In [ ]:
original_crop = 96
for newcrop in range(original_crop,2,-1):
    if newcrop <= min(minshapes):
        crop_shapes = tuple([newcrop]*3)
        break
crop_shapes

In [ ]:
val_pct = 0.2
val_split = int(val_pct*len(labels))
possible_labels = sorted(list(set(labels)))
proportion_in_labels = np.array([sum(labels==i)/len(labels) for i in possible_labels])
val_per_labels = [int(l*val_split) for l in proportion_in_labels]
proportion_in_labels, val_per_labels

In [ ]:
label_weigths = 1/proportion_in_labels[labels]

In [ ]:
val_idx = np.concatenate([np.random.choice([i for i, l in enumerate(labels) if l==p], c, replace=False) for p,c in zip(possible_labels, val_per_labels)])
in_val = np.isin(np.arange(len(labels)),val_idx)
in_train = ~in_val
train_idx = np.arange(len(labels))[in_train]
val_idx

In [ ]:
batch_size = 4#3
while (len(images)-val_split)%batch_size==1 or val_split%batch_size==1:
    batch_size +=1
    print("Changing batch size so that no batch has size 1")
batch_size

In [ ]:
# Define transforms
train_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize(crop_shapes), RandRotate90()])

val_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize(crop_shapes)])

# Define nifti dataset, data loader
check_ds = ImageDataset(image_files=images, labels=labels, transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=3, num_workers=2, pin_memory=pin_memory)

im, label = monai.utils.misc.first(check_loader)
print(type(im), im.shape, label, label.shape)

# create a training data loader
train_ds = ImageDataset(image_files=images[train_idx].tolist(), labels=labels[train_idx].tolist(), transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=pin_memory)

# create a validation data loader
val_ds = ImageDataset(image_files=images[val_idx].tolist(), labels=labels[val_idx].tolist(), transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=2, pin_memory=pin_memory)

In [ ]:
len(images)

In [ ]:
len(train_ds), len(val_ds)

In [ ]:
len(train_loader), len(val_loader)

In [ ]:
metrics_fns = {
    s.__name__.split("_score")[0]: s for s in [f1_score, accuracy_score, balanced_accuracy_score]
}
val_metric = "balanced_accuracy"
metrics_fns

In [ ]:
# Create DenseNet121, CrossEntropyLoss and Adam optimizer
model = monai.networks.nets.DenseNet(spatial_dims=3, in_channels=1, out_channels=num_labels).to(device)

loss_function = torch.nn.CrossEntropyLoss(torch.tensor(1/proportion_in_labels, device=device, dtype=torch.float32))
# loss_function = torch.nn.BCEWithLogitsLoss()  # also works with this data

optimizer = torch.optim.Adam(model.parameters(), 1e-4)

# start a typical PyTorch training
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
writer = SummaryWriter(f"v2_runs/{datetime.datetime.now():%Y-%m-%d_%H:%M:%S}_{socket.gethostname()}")
max_epochs = 256

for epoch in range(max_epochs):
    try:
        print("-" * 10)
        print(f"epoch {epoch + 1}/{max_epochs}")
        model.train()
        epoch_loss = 0
        step = 0

        for batch_data in train_loader:
            step += 1
            b_inputs, b_labels = batch_data[0].to(device), batch_data[1].to(device)
            optimizer.zero_grad()
            outputs = model(b_inputs)
            loss = loss_function(outputs, b_labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            epoch_len = len(train_ds) // train_loader.batch_size
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

        if (epoch + 1) % val_interval == 0:
            model.eval()

            num_correct = 0.0
            metric_count = 0
            v_pred, v_label = get_predicted_values(model, val_loader)
            cm_counts = get_cm(v_pred, v_label, num_labels)

            all_metrics = {
                metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
            }
            metric = all_metrics[val_metric]
            metric_values.append(metric)

            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), "v2_best_metric_model_classification3d_array.pth")
                print("saved new best metric model")

            print(f"Current epoch: {epoch+1} current {val_metric}: {metric:.4f} ")
            print(f"Best {val_metric}: {best_metric:.4f} at epoch {best_metric_epoch}")
            for metric in all_metrics:
                writer.add_scalar(f"val_{metric}", all_metrics[metric], epoch + 1)
            cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)
            print(cm_pct)
            for i in range(num_labels):
                for j in range(num_labels):
                    writer.add_scalar(f"count l{i}_p{j}",cm_counts[i,j], epoch + 1)
                    writer.add_scalar(f"pct l{i}_p{j}",cm_pct[i,j], epoch + 1)
    except KeyboardInterrupt: 
        break

print(f"Training completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
writer.close()

# Occlusion sensitivity
One method for trying to visualise why the network made a given prediction is occlusion sensitivity. We occlude part of the image, and see how the probability of a given prediction changes. We then iterate over the image, moving the occluded portion as we go, and in doing so we build up a sensitivity map detailing which areas were the most important in making the decision.

#### Bounds
If we were to test the occlusion centred on all voxels in our image, we would have to do `torch.prod(im.shape) = 96^3 = ~1e6` predictions. We can use the bounding box to only to the estimations in a region of interest, for example over one slice.

To do this, we simply give the bounding box as `(minC,maxC,minD,maxD,minH,maxH,minW,maxW)`. We can use `-1` for any value to use its full extent (`0` and `im.shape-1` for min's and max's, respectively).

#### Output
The output image in this example will look fairly bad, since our network hasn't been trained for very long. Training for longer should improve the quality of the occlusion map.

In [ ]:
# create a validation data loader
test_ds = ImageDataset(image_files=test_images, labels=test_labels, transform=val_transforms)
test_loader = DataLoader(test_ds, batch_size=2, num_workers=2, pin_memory=torch.cuda.is_available())

In [ ]:
len(test_images)

In [ ]:
v_label, v_pred = get_predicted_values(model, train_loader)
conf_matrix = get_cm(v_label, v_pred, num_labels)
all_metrics = {
    metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
}
conf_matrix, all_metrics

In [ ]:
v_label, v_pred = get_predicted_values(model, val_loader)
conf_matrix = get_cm(v_label, v_pred, num_labels)
all_metrics = {
    metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
}
conf_matrix, all_metrics

In [ ]:
v_label, v_pred = get_predicted_values(model, test_loader)
conf_matrix = get_cm(v_label, v_pred, num_labels)
all_metrics = {
    metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
}
conf_matrix, all_metrics

In [ ]:
occ_loader = DataLoader(test_ds, batch_size=1, num_workers=2, pin_memory=pin_memory)
itera = iter(occ_loader)


def get_next_im():
    test_data = next(itera)
    return test_data[0].to(device), test_data[1].unsqueeze(0).to(device)


def plot_occlusion_heatmap(im, heatmap):
    plt.subplots(1, 2)
    plt.subplot(1, 2, 1)
    plt.imshow(np.squeeze(im.cpu()))
    plt.colorbar()
    plt.subplot(1, 2, 2)
    plt.imshow(heatmap)
    plt.colorbar()
    plt.show()

In [ ]:
# Get a random image and its corresponding label
img, label = get_next_im()

In [ ]:
# Get the occlusion sensitivity map
occ_sens = monai.visualize.OcclusionSensitivity(nn_module=model, mask_size=12, n_batch=10, stride=12)
# Only get a single slice to save time.
# For the other dimensions (channel, width, height), use
# -1 to use 0 and img.shape[x]-1 for min and max, respectively
depth_slice = img.shape[2] // 2
occ_sens_b_box = [depth_slice-1, depth_slice, -1, -1, -1, -1]

occ_result, _ = occ_sens(x=img, b_box=occ_sens_b_box)
occ_result = occ_result[0, label.argmax().item()][None]

fig, axes = plt.subplots(1, 2, figsize=(25, 15), facecolor="white")

for i, im in enumerate([img[:, :, depth_slice, ...], occ_result]):
    cmap = "gray" if i == 0 else "jet"
    ax = axes[i]
    im_show = ax.imshow(np.squeeze(im[0][0].detach().cpu()), cmap=cmap)
    ax.axis("off")
    fig.colorbar(im_show, ax=ax)

## Cleanup data directory

Remove directory if a temporary was used.

In [ ]:
if directory is None:
    shutil.rmtree(root_dir)